In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 1: Generate Synthetic DHT11-like Data
np.random.seed(42)
data_size = 300
temperature = np.random.uniform(15, 35, size=data_size)  # Celsius
humidity = np.random.uniform(30, 90, size=data_size)     # Percent

# Step 2: Create Rule-Based Labels
labels = []
for t, h in zip(temperature, humidity):
    if t < 20 or h < 40:
        labels.append("Cold")
    elif 20 <= t <= 28 and 40 <= h <= 70:
        labels.append("Comfort")
    else:
        labels.append("Hot")

# Step 3: Create DataFrame
df = pd.DataFrame({
    "temperature": temperature,
    "humidity": humidity,
    "label": labels
})

# Step 4: Encode Labels
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])

# Step 5: Prepare Features and Target
X = df[["temperature", "humidity"]].values
y = tf.keras.utils.to_categorical(df["label_encoded"].values, num_classes=3)

# Step 6: Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 7: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Step 8: Build TensorFlow Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 9: Train Model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=1)

# Step 10: Evaluate and Save Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

model.save("model.h5")
print("model.h5 saved successfully!")


IndentationError: expected an indented block after 'for' statement on line 13 (2699636406.py, line 14)

In [ ]:
# Step 11: Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Step 12: Save the .tflite file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved model.tflite for ESP32")


In [ ]:
def bin_to_c_array(input_file, output_file, array_name="model_tflite"):
    with open(input_file, "rb") as f:
        data = f.read()

    with open(output_file, "w") as f:
        f.write(f"const unsigned char {array_name}[] = {{\n")
        for i in range(0, len(data), 12):
            chunk = data[i:i+12]
            line = ", ".join(f"0x{b:02x}" for b in chunk)
            f.write("  " + line + ",\n")
        f.write("};\n")
        f.write(f"const unsigned int {array_name}_len = {len(data)};\n")

# Usage:
bin_to_c_array("model.tflite", "model_data.h")
